# How's The Weather Out There?

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import API key
from config import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [ ]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [ ]:
# Create a list to store the json response for each city
responses = []

# Create base url
base_url = "http://api.openweathermap.org/data/2.5/weather?q="

city_count = 0
for city in cities:
    url = base_url + f"{city}&units=metric&appid={weather_api_key}"
    city_data = requests.get(url).json()
    if city_data['cod'] == '404':
        print('City not found. Skipping...')
    else:
        responses.append(city_data)
        city_count += 1
        print(f"Processing record {city_count} | {city_data['name']}")

## Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [ ]:
# Create lists to store values
found_cities = []
lats = []
lngs = []
max_temps = []
humidities = []
clouds = []
wind_speeds = []
countries = []
dates = []


for i in range(city_count):
    response = responses[i]
    found_cities.append(response['name'])
    lats.append(response['coord']['lat'])
    lngs.append(response['coord']['lon'])
    max_temps.append(response['main']['temp_max'])
    humidities.append(response['main']['humidity'])
    clouds.append(response['clouds']['all'])
    wind_speeds.append(response['wind']['speed'])
    countries.append(response['sys']['country'])
    dates.append(response['dt'])
    
city_weather_data = pd.DataFrame({'City': found_cities,
                                  'Lat': lats,
                                  'Lng': lngs,
                                  'Max Temp': max_temps,
                                  'Humidity': humidities,
                                  'Cloudiness': clouds,
                                  'Wind Speed': wind_speeds,
                                  'Country': countries,
                                  'Date': dates})
city_weather_data.head()

In [ ]:
city_weather_data.describe()

In [ ]:
city_weather_data.to_csv(output_data_file, index=False)

## Inspect the data and remove the cities where the humidity > 100%.
----

Based on the summary table above, there are no cities where humidity > 100%!

## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

In [ ]:
# Set x and y values
city_weather_data.plot('Lat', 'Max Temp', kind='scatter', xlabel='Latitude', ylabel='Max Temperature (C)', title='Max Temperature Vs Latitude', grid=True)
plt.show()

## Latitude vs. Humidity Plot

In [ ]:
city_weather_data.plot('Lat', 'Humidity', kind='scatter', xlabel='Latitude', ylabel='Humidity (%)', title='Humidity Vs Latitude', grid=True)
plt.show()

## Latitude vs. Cloudiness Plot

In [ ]:
city_weather_data.plot('Lat', 'Cloudiness', kind='scatter', xlabel='Latitude', ylabel='Cloudiness (%)', title='Cloudiness Vs Latitude', grid=True)
plt.show()

## Latitude vs. Wind Speed Plot

In [ ]:
city_weather_data.plot('Lat', 'Wind Speed', kind='scatter', xlabel='Latitude', ylabel='Wind Speed (mph)', title='Wind Speed Vs Latitude', grid=True)
plt.show()

## Linear Regression

In [ ]:
# split data into northern and southern hemispheres
northern_hem_cities = city_weather_data.loc[(city_weather_data['Lat'] > 0)]
southern_hem_cities = city_weather_data.loc[(city_weather_data['Lat'] < 0)]

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
xvalues = northern_hem_cities['Lat']
yvalues = northern_hem_cities['Max Temp']
slope, intercept, r, p, std_err = linregress(xvalues, yvalues)
fit = slope*xvalues + intercept
fit_eq = f'y = {round(slope, 2)}x + {round(intercept, 2)}'

print(f"The R-value is {r}")
northern_hem_cities.plot('Lat', 'Max Temp', kind='scatter', xlabel='Latitude', ylabel='Max Temperature (C)', title='Max Temperature Vs Latitude')
plt.plot(xvalues, fit, "-", color='r')
plt.annotate(fit_eq, (10,-10), color='r')
plt.show()

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
xvalues = southern_hem_cities['Lat']
yvalues = southern_hem_cities['Max Temp']
slope, intercept, r, p, std_err = linregress(xvalues, yvalues)
fit = slope*xvalues + intercept
fit_eq = f'y = {round(slope, 2)}x + {round(intercept, 2)}'

print(f"The R-value is {r}")
southern_hem_cities.plot('Lat', 'Max Temp', kind='scatter', xlabel='Latitude', ylabel='Max Temperature (C)', title='Max Temperature Vs Latitude')
plt.plot(xvalues, fit, "-", color='r')
plt.annotate(fit_eq, (-50,30), color='r')
plt.show()

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
xvalues = northern_hem_cities['Lat']
yvalues = northern_hem_cities['Humidity']
slope, intercept, r, p, std_err = linregress(xvalues, yvalues)
fit = slope*xvalues + intercept
fit_eq = f'y = {round(slope, 2)}x + {round(intercept, 2)}'

print(f"The R-value is {r}")
northern_hem_cities.plot('Lat', 'Humidity', kind='scatter', xlabel='Latitude', ylabel='Humidity (%)', title='Humidity Vs Latitude')
plt.plot(xvalues, fit, "-", color='r')
plt.annotate(fit_eq, (0,34), color='r')
plt.show()

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
xvalues = southern_hem_cities['Lat']
yvalues = southern_hem_cities['Humidity']
slope, intercept, r, p, std_err = linregress(xvalues, yvalues)
fit = slope*xvalues + intercept
fit_eq = f'y = {round(slope, 2)}x + {round(intercept, 2)}'

print(f"The R-value is {r}")
southern_hem_cities.plot('Lat', 'Humidity', kind='scatter', xlabel='Latitude', ylabel='Humidity (%)', title='Humidity Vs Latitude')
plt.plot(xvalues, fit, "-", color='r')
plt.annotate(fit_eq, (-52,42), color='r')
plt.show()

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
xvalues = northern_hem_cities['Lat']
yvalues = northern_hem_cities['Cloudiness']
slope, intercept, r, p, std_err = linregress(xvalues, yvalues)
fit = slope*xvalues + intercept
fit_eq = f'y = {round(slope, 2)}x + {round(intercept, 2)}'

print(f"The R-value is {r}")
northern_hem_cities.plot('Lat', 'Cloudiness', kind='scatter', xlabel='Latitude', ylabel='Cloudiness (%)', title='Cloudiness Vs Latitude')
plt.plot(xvalues, fit, "-", color='r')
plt.annotate(fit_eq, (51,44), color='r')
plt.show()

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
xvalues = southern_hem_cities['Lat']
yvalues = southern_hem_cities['Cloudiness']
slope, intercept, r, p, std_err = linregress(xvalues, yvalues)
fit = slope*xvalues + intercept
fit_eq = f'y = {round(slope, 2)}x + {round(intercept, 2)}'

print(f"The R-value is {r}")
southern_hem_cities.plot('Lat', 'Cloudiness', kind='scatter', xlabel='Latitude', ylabel='Cloudiness (%)', title='Cloudiness Vs Latitude')
plt.plot(xvalues, fit, "-", color='r')
plt.annotate(fit_eq, (-52,53), color='r')
plt.show()

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
xvalues = northern_hem_cities['Lat']
yvalues = northern_hem_cities['Wind Speed']
slope, intercept, r, p, std_err = linregress(xvalues, yvalues)
fit = slope*xvalues + intercept
fit_eq = f'y = {round(slope, 2)}x + {round(intercept, 2)}'

print(f"The R-value is {r}")
northern_hem_cities.plot('Lat', 'Wind Speed', kind='scatter', xlabel='Latitude', ylabel='Wind Speed (m/s)', title='Wind Speed Vs Latitude')
plt.plot(xvalues, fit, "-", color='r')
plt.annotate(fit_eq, (5,12), color='r')
plt.show()

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
xvalues = southern_hem_cities['Lat']
yvalues = southern_hem_cities['Wind Speed']
slope, intercept, r, p, std_err = linregress(xvalues, yvalues)
fit = slope*xvalues + intercept
fit_eq = f'y = {round(slope, 2)}x + {round(intercept, 2)}'

print(f"The R-value is {r}")
southern_hem_cities.plot('Lat', 'Wind Speed', kind='scatter', xlabel='Latitude', ylabel='Wind Speed (m/s)', title='Wind Speed Vs Latitude')
plt.plot(xvalues, fit, "-", color='r')
plt.annotate(fit_eq, (-55,8), color='r')
plt.show()